In [ ]:
from src.model import LIFNetwork as lif
import numpy as np
from importlib import reload
import os

reload(lif)

In [ ]:
LIF = lif.LIF_Network(n_neurons=5)
LIF.simulate(epoch_current_input=None)

In [ ]:
vec_a = np.random.randint(low=0, high=5, size=3)
mat_a = np.random.randint(low=0, high=5, size=(3, 3))
mat_b = np.random.randint(low=0, high=5, size=(3, 3))

print(vec_a)
print()
print(mat_a)
print()
print(mat_b)
print()

print(np.matmul(vec_a, mat_a))
print(np.matmul(mat_a, vec_a))

## Okay, so the order of matmul does matter, and it is okay to switch them.

In [ ]:
print(2 * mat_b)

In [ ]:
os.mkdir("./test")

In [ ]:
## Library imports
import numpy as np
from src.model import LIFNetwork as lif
import matplotlib.pyplot as plt
import os
from datetime import datetime 
from pathlib import Path
time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

## Assumptions in the following simulation: 
## - We assume that the model is correct, which is not the case and some of the
##   discovered issues have been logged in github issues pending to be fixed.
## - We assume each finger is receiving an equal duration of simulation by
##   dividing the total simulation duration by 10 (fingers). In future 
##   experiments, we may vary the division of simulation duration between the 
##   10 fingers.
## - 

## Experiment Parameters (Change the following parameters to experiment!)
glove_stim_freq = 50         # Hz
n_finger = 10                # Number of fingers
n_neurons = 200              # Number of neurons
proba_conn = 0.2             # Probability of connection for finger-to-neuron 
glove_stim_duration = 20000  # milliseconds
# NOTE (Tony): The kuramato method is weird and is pending to be fixed. 
#   Setting to None allows for 1000/0.1 = 1000 sections for the period 
#   calculation and thus is more accurate.
kuramato_period = None
kuramato_lookBack = 1000     # milliseconds

## Tracker Variables
x = []  # Tracker of variable o ---> ??? WHY?
track_post_sim_W_running_mean = []  # Tracker of running average of mean_W2
track_kuramato_order = []  # Tracker of Kuramato order 
track_kuramato_order_running_mean = [] # Tracker of running average of Kuramato order
o = 0   # Counter of the number-th run (why not just use the i from the for loop?)
track_pre_sim_W_mean = []
track_post_sim_W_mean = []  # Holder of mean_W2

finger_stim = np.zeros([glove_stim_duration, n_finger])    # Tracker of finger stimulation over time
finger_neuron_conn = np.zeros([n_finger, n_neurons])       # Tracker of finger-Neuron connection stimulation
neuron_input = np.zeros([glove_stim_duration, n_neurons])  # Tracker of input to neuron (generated by glove stimulation)

## LIF instantiation
LIF = lif.LIF_Network(n_neurons = n_neurons)
dt = LIF.dt  # Time step

## Stimulation variables
inter_stim_time = int(1/glove_stim_freq * 1000)               # milliseconds
stim_duration_per_finger = int(glove_stim_duration/n_finger)  # Total stimulation length per finger - Assuming each finger receives equal stimulation? 

## Finger stimulation order
stim_order = np.arange(n_finger)
np.random.shuffle(stim_order)


## Generating finger to neuron stimulation, **based on probability**
finger_neuron_conn = np.random.rand(n_finger, n_neurons)
finger_neuron_conn = (finger_neuron_conn > proba_conn)


## Update tracker to mark whether a neuron receives current from finger-stimulation
for finger_idx in range(0, n_finger):
  for time_idx in range(stim_duration_per_finger * (finger_idx), 
                        stim_duration_per_finger * (finger_idx + 1), 
                        inter_stim_time): 
    if time_idx < glove_stim_duration:
      finger_stim[time_idx, finger_idx] = 1          # Update the finger stimulation tracker
      neuron_input[time_idx, finger_neuron_conn[finger_idx, :]] = 1        # Based on finger_neuron_conn, marking whether a neuron receives input form the finger stimulation


## RUNNING GLOVE STIMULATION SIMULATION ##
euler_steps = int(glove_stim_duration / dt)
for step in range(euler_steps):
    ## Pre simulation network connection weights
    # pre_sim_W = np.copy(LIF.network_W)
    # pre_sim_W_mean = pre_sim_W[pre_sim_W>0].mean()
    pre_sim_W_mean = LIF.network_W[LIF.network_W > 0].mean()
    track_pre_sim_W_mean.append(pre_sim_W_mean)

    # NOTE (Tony):
    # The second argument of plotter2 is sim_duration, feeding directly 
    # into the LIF_Network.simulate() argument. The original argument name
    # was ambiguous and thus may have led to the misunderstanding of 
    # LIF_Network.stimulate()'s second argument as number of Euler steps.
    
    # FIXME (Tony): This method's implementation of the "input matrix" (i.e., 
    # epoch_current_input) is incorrect, and the method needs to be fixed
    # before simulation can be ran here. Github issue has been submitted.
    LIF.simulate(sim_duration = glove_stim_duration, 
                 epoch_current_input = None)
    
    ## Post simulation network connection weights
    # post_sim_W = LIF.network_W
    # post_sim_W_mean = post_sim_W[post_sim_W > 0].mean()
    post_sim_W_mean = LIF.network_W[LIF.network_W > 0].mean()
    track_post_sim_W_mean.append(post_sim_W_mean)

    ## Network connection weight difference (post - pre)
    mean_W_diff = post_sim_W_mean - pre_sim_W_mean
    
    ## Calculate post simulation kuramato-order 
    kuramato_order = LIF.kuramato(period = kuramato_period, 
                                  lookBack = kuramato_lookBack)
    track_kuramato_order.append(kuramato_order)
    
    # ## Calculate post simulation W running mean
    # running_W_mean = (sum(track_post_sim_W_mean)
    #                 / len(track_post_sim_W_mean))
    # track_post_sim_W_running_mean.append(running_W_mean)
    
    # ## Calculate post simulation kuramato-order running mean
    # running_kuramato_mean = (sum(track_kuramato_order) 
    #                          / len(track_kuramato_order))
    # track_kuramato_order_running_mean.append(running_kuramato_mean)

    ## Plot graph
    with plt.ioff():    
      fig, ax = plt.subplots()
      # ax.plot(x, track_post_sim_W_running_mean,
      #         linestyle = "--", marker = ".", 
      #         label='Running W-mean')
      # ax.plot(x, track_kuramato_order_running_mean,
      #         linestyle = "--", marker = ".",
      #         label='Kuramato Running mean')
      ax.plot(track_post_sim_W_mean, 
              linestyle = "--", marker = ".", label = "Post Stim network weight mean")
      ax.plot(track_kuramato_order,
              linestyle = "--", marker = ".", label = "Post Stim Kuramato order")
      ax.set_xlim(0, )
      ax.set_ylim(0, 1)
      ax.set_xlabel("Glove Stimulation (Euler) Step")
      ax.set_title(f"Kuramato Order & PostStim Network-Weight_Mean at each stim step \n step {step}")
      ax.legend()

      dir_name = f"./plot_exports_{time_str}"
      file_name = f"{dir_name}/plot_{time_str}"

      if not(os.path.exists(dir_name)):
          os.mkdir(dir_name)

      ## Show and save image every 10 step
      if (step % 10 == 0):
          file_name_path = Path(f"{file_name}_step_{step}.png")
          print(file_name_path)
          plt.savefig(file_name_path, dpi=150)